In [6]:
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.master("local[*]").appName("test").config("spark.driver.bindAddress","127.0.0.1").getOrCreate()

24/02/25 15:15:06 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz

In [81]:
import gzip
with gzip.open('fhv_tripdata_2019-10.csv.gz', 'rb') as gz:
    file_content = gz.read()
    with open("fhv_tripdata_2019-10.csv","wb") as csv:
        csv.write(file_content);

In [82]:
# Question 1. Spark version
spark.version

'3.5.0'

In [83]:
fhv_df = spark.read.option("header", "true").csv('fhv_tripdata_2019-10.csv')

In [84]:
# Question 2. FHV October 2019 partition size
fhv_df = fhv_df.repartition(6)
fhv_df.write.parquet('fhv_data', mode="overwrite")

import glob, os
count = 0
size = 0
for file in glob.glob("fhv_data/*.parquet"):
    size += os.path.getsize(file)/(1024*1024)
    count += 1
size//count

6.0

In [85]:
# Question 3. Count records on 15th of October
from pyspark.sql.functions import *
fhv_df = fhv_df.withColumn("pickup_time",to_timestamp("pickup_datetime")).withColumn("dropoff_time",to_timestamp("dropOff_datetime"))
fhv_df.createOrReplaceTempView("fhv")
spark.sql("SELECT pickup_time FROM fhv WHERE pickup_time >= '2019-10-15 00:00:00' AND pickup_time <= '2019-10-15 23:59:59'").count()

62610

In [86]:
# Question 4. The longest trip
spark.sql("SELECT date_diff(dropoff_time,pickup_time)*24 as diff  FROM fhv ORDER BY diff DESC LIMIT 1").show()

+------+
|  diff|
+------+
|631152|
+------+



In [87]:
# Question 6. Least frequent pickup location zone
zones_df = spark.read.option("header", "true").csv('taxi+_zone_lookup.csv')
zones_df.createOrReplaceTempView("zones")
spark.sql("SELECT Zone,SUM(1) as cnt FROM fhv JOIN zones ON PUlocationId=LocationID GROUP BY Zone ORDER BY cnt ASC LIMIT 1").show()

+-----------+---+
|       Zone|cnt|
+-----------+---+
|Jamaica Bay|  1|
+-----------+---+

